In [1]:
# =========================
# SVR – Debug Tuning-Pipeline (kleines, schnelles Grid)
# =========================
# - Master-Schalter:
#     USE_DYNAMIC_FI_PIPELINE = False -> Full FE (ifo + optionale Target-Blöcke)
#     USE_DYNAMIC_FI_PIPELINE = True  -> Dynamic FI (rolling Feature Importance Top-N)
# - Modell: src.models.SVR.ForecastModel
# - GlobalConfig(preset="debug") -> FAST_DEBUG-Splits gemäß config.py

import os, sys
from pathlib import Path
import numpy as np
import pandas as pd
from itertools import product

# --- 1) Pfad-Setup (wie bei ET/EN/LGBM) ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(6):
        if (cur / "src").exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT =", PROJECT_ROOT)

# --- 2) Repo-Imports ---
from src.config import (
    GlobalConfig,
    DEFAULT_CORR_SPEC,    # expanding
    EWMA_CORR_SPEC,       # ewma
    outputs_for_model,
)
from src.tuning import run_stageA, run_stageB
from src.io_timesplits import (
    load_target,
    load_ifo_features,
    load_full_lagged_features,
    load_rolling_importance,
)
from src.models.SVR import ForecastModel  # SVR-Wrapper

# --- 3) Master-Schalter & Meta ---
USE_DYNAMIC_FI_PIPELINE = False   # Debug: Full FE schneller
SEED   = 42

MODEL_NAME = "svr_debug_dynamic_fi" if USE_DYNAMIC_FI_PIPELINE else "svr_debug"
outputs_for_model(MODEL_NAME)
print(f"Modell {MODEL_NAME} wird getunt.")

# --- 4) Daten laden ---
y = load_target()             # ΔIP (DatetimeIndex)
X_ifo = load_ifo_features()   # ifo features (für Full FE)

FI_BASE_DIR = PROJECT_ROOT / "outputs" / "feature_importance" / "outputs_no_missing"

if USE_DYNAMIC_FI_PIPELINE:
    try:
        X_full_lagged = load_full_lagged_features(base_dir=FI_BASE_DIR)
        rolling_imp   = load_rolling_importance(base_dir=FI_BASE_DIR)
        idx_fi = y.index.intersection(X_full_lagged.index).intersection(rolling_imp.index)
        y_fi          = y.loc[idx_fi]
        X_full_lagged = X_full_lagged.loc[idx_fi]
        rolling_imp   = rolling_imp.loc[idx_fi]
        print("Dynamic-FI Daten gefunden. Shapes:", X_full_lagged.shape, rolling_imp.shape, y_fi.shape)
    except FileNotFoundError as e:
        print("FEHLER beim Laden der Dynamic-FI Artefakte:", e)
        print("Bitte erst die FI-Jobs laufen lassen (feature_importance.ipynb).")
        raise
else:
    # Full FE: Indizes alignen
    idx_common = y.index.intersection(X_ifo.index)
    y    = y.loc[idx_common]
    X_ifo = X_ifo.loc[idx_common]
    X_full_lagged = rolling_imp = y_fi = None
    print("Full-FE Daten geladen. Shapes:", X_ifo.shape, y.shape)

# --- 5) Base-Config (Debug-Splits) ---
def base_cfg_debug() -> GlobalConfig:
    # Jede preset != "thesis" triggert in config.py den FAST_DEBUG-Fallback
    cfg = GlobalConfig(preset="debug")
    # Policy wie in der Arbeit, aber die Splits sind kurz/leicht
    cfg.policy_window   = 12
    cfg.policy_decay    = 0.95
    cfg.policy_gain_min = 0.03
    cfg.policy_cooldown = 2
    return cfg

cfg0 = base_cfg_debug()

# --- 6) Corr-Helper ---
def make_corr_spec(kind: str) -> dict:
    if kind == "expanding":
        return dict(DEFAULT_CORR_SPEC)
    elif kind == "ewm":
        return dict(EWMA_CORR_SPEC)
    else:
        raise ValueError("kind must be 'expanding' or 'ewm'")

# --- 7) Kleine Grids -------------------------------------------------
if USE_DYNAMIC_FI_PIPELINE:
    # --------- Dynamic FI (nur Modell-HPs + n_features_to_use) ----------
    print("Erstelle HP-Grid für 'Dynamic FI' (Debug klein)...")

    N_FEATURES_TO_USE = 10  # kleiner als im Thesis-Lauf

    # Sehr kleines, schnelles SVR-Grid
    kernel_list  = ["linear", "rbf"]
    C_list       = [0.5, 1.0]
    epsilon_list = [0.1]
    gamma_list   = ["scale"]  # nur relevant für rbf, aber ok
    weight_opts  = [{"sample_weight_decay": None}]

    def build_model_grid_dynamic_fi():
        grid = []
        for k, C, eps, g in product(kernel_list, C_list, epsilon_list, gamma_list):
            hp = {
                "kernel": k,
                "C": C,
                "epsilon": eps,
                "gamma": g,
                "seed": SEED,
                "n_features_to_use": N_FEATURES_TO_USE,
                **weight_opts[0],
            }
            grid.append(hp)
        return grid

    model_grid = build_model_grid_dynamic_fi()

else:
    # ----------------- Full FE (Debug klein) ---------------------
    print("Erstelle HP-Grid für 'Full FE' (Debug klein)...")

    # A) FE/DR-Listen stark reduziert
    corr_options = [
        ("expanding", make_corr_spec("expanding")),
    ]
    lag_candidates_list   = [(1, 3, 12)]
    top_k_lags_list       = [1]
    use_rm3_list          = [True]
    k1_topk_list          = [100]
    redundancy_param_list = [0.90]
    dr_options_list       = [
        {"dr_method": "none"},
        {"dr_method": "pca", "pca_var_target": 0.90, "pca_kmax": 20},
    ]

    # B) Sehr kleines SVR-HP-Grid
    kernel_list  = ["linear", "rbf"]
    C_list       = [0.5, 1.0]
    epsilon_list = [0.1]
    gamma_list   = ["scale"]  # für rbf
    # Target Blocks & Weighting (reduziert)
    target_block_options = [None]
    weight_opts          = [{"sample_weight_decay": None}]

    def build_model_grid_full_fe():
        hp_grid = []
        fe_lists = [
            lag_candidates_list,      # lags
            top_k_lags_list,          # k_lags
            use_rm3_list,             # rm3
            k1_topk_list,             # k1
            redundancy_param_list,    # red
            dr_options_list,          # dr_opt
        ]
        for (corr_tag, corr_spec) in corr_options:
            for (lags, k_lags, rm3, k1, red, dr_opt) in product(*fe_lists):
                for k, C, eps, g in product(kernel_list, C_list, epsilon_list, gamma_list):
                    for block_set in target_block_options:
                        hp = {
                            # FE
                            "lag_candidates": lags,
                            "top_k_lags_per_feature": k_lags,
                            "use_rm3": rm3,
                            "k1_topk": k1,
                            "redundancy_param": red,
                            **dr_opt,
                            "corr_tag": corr_tag,
                            "corr_spec": corr_spec,
                            # Modell
                            "kernel": k,
                            "C": C,
                            "epsilon": eps,
                            "gamma": g,
                            "seed": SEED,
                            # Target Blocks / Weights
                            "target_block_set": block_set,
                            **weight_opts[0],
                        }
                        hp_grid.append(hp)
        return hp_grid

    model_grid = build_model_grid_full_fe()

print("HP-Kombinationen (Debug):", len(model_grid))
print("Erstes HP-Set:", model_grid[0] if model_grid else "Grid ist leer")

# --- 8) Stage A/B Lauf (Debug) ------------------------------------------
if model_grid:
    if USE_DYNAMIC_FI_PIPELINE:
        shortlist = run_stageA(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            model_grid=model_grid,
            X=X_ifo,   # Platzhalter
            y=y_fi,
            cfg=cfg0,
            keep_top_k_final=min(3, len(model_grid)),
            min_survivors_per_block=max(1, len(model_grid)//3),
            X_full_lagged=X_full_lagged,
            rolling_imp=rolling_imp,
        )
        run_stageB(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            shortlist=shortlist,
            X=X_ifo,  # Platzhalter
            y=y_fi,
            cfg=cfg0,
            X_full_lagged=X_full_lagged,
            rolling_imp=rolling_imp,
        )
    else:
        shortlist = run_stageA(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            model_grid=model_grid,
            X=X_ifo,
            y=y,
            cfg=cfg0,
            keep_top_k_final=min(3, len(model_grid)),
            min_survivors_per_block=max(1, len(model_grid)//3),
        )
        run_stageB(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            shortlist=shortlist,
            X=X_ifo,
            y=y,
            cfg=cfg0,
        )
else:
    print("Keine gültigen HP-Kombinationen, Stages übersprungen.")

print(f"\nDone. Check outputs/stageA|stageB/{MODEL_NAME} for results.")


PROJECT_ROOT = /Users/jonasschernich/Documents/Masterarbeit/Code
Modell svr_debug wird getunt.
INFO in load_ifo_features: Renaming columns to ensure validity.
Full-FE Daten geladen. Shapes: (407, 2160) (407,)
Erstelle HP-Grid für 'Full FE' (Debug klein)...
HP-Kombinationen (Debug): 8
Erstes HP-Set: {'lag_candidates': (1, 3, 12), 'top_k_lags_per_feature': 1, 'use_rm3': True, 'k1_topk': 100, 'redundancy_param': 0.9, 'dr_method': 'none', 'corr_tag': 'expanding', 'corr_spec': {'mode': 'expanding', 'window': None, 'lam': None}, 'kernel': 'linear', 'C': 0.5, 'epsilon': 0.1, 'gamma': 'scale', 'seed': 42, 'target_block_set': None, 'sample_weight_decay': None}
[Stage A] Using FULL FE pipeline (Gleis 1 & 2).
[Stage A][Block 1] train_end=48, OOS=49-60 | configs=8
  - Config 1/8
    · Month 5/12 processed | running...RMSE=27.4772
    · Month 10/12 processed | running...RMSE=29.0096
    · Month 12/12 processed | running...RMSE=29.0921
  - Config 2/8
    · Month 5/12 processed | running...RMSE=27.47